In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
from time import time
import datetime
import random

In [2]:
def valid_solution():
    x_valido = {}
    
    # Inicialmente matriz c con costos muy elevados
    c_disponibles = np.full((m, n), 1e5)
    
    for j in range(n):
        unos = []

        for i in range(m):
            # Asignamos inicialmente valor cero a cada coordenada 
            x_valido[i, j] = 0
            
            if y[i].X == 1:
                # Instalación i está disponible por lo tanto el costo para la matriz c toma su valor original
                c_disponibles[i, j] = c[i, j]
            
            if x[i, j].X == 1:
                unos.append(((i, j), c[i, j]))


        if len(unos) == 0:
            # cliente no asignado a instalación
            coordenadas_min = np.where(c_disponibles[:,j] == c_disponibles[:,j].min())[0]
            i_seleccionado = random.choice(coordenadas_min)
        
        elif len(unos) == 1:
            # Cliente asignado exactamente a una instalación
            i_seleccionado = unos[0][0][0]

        elif len(unos) > 1:
            # Cliente asignado a más de una instalación        
            unos.sort(key = lambda x:x[1])
            coordenadas_posibles = [element[0][0] for element in unos if element[1] == unos[0][1]]
            i_seleccionado = random.choice(coordenadas_posibles)


        # Asignamos una y solo una instalación al cliente j
        x_valido[i_seleccionado, j] = 1
    
    return x_valido

In [3]:
def upper_bound(x_valido):
    suma = 0
    for i in range(m):
        suma += f[i] * y[i].X
        for j in range(n):
            suma += c[i, j] * x_valido[i, j]
    return suma

In [4]:
NITER = 100

# Acá se definen los datos
m = 50
n = 2000
M = range(m)
N = range(n)

np.random.seed(71096)

c = np.random.randint(3,7,size=(m,n))
f = np.random.randint(10,30,size=m)

In [5]:
# Se hacen las definiciones iniciales de los modelos

model = gp.Model('Location and Asignment with Lagrangian Relaxation')
model.setParam('OutputFlag', False) # turns off solver chatter

modelf = gp.Model('Location and Asignment with Lagrangian Relaxation')
modelf.setParam('OutputFlag', True) # turns on solver chatter

Using license file /home/tomas/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-25
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


In [33]:
# Esta es la definición de variables y restricciones del problema completo original
xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")


modelf.setObjective((sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M)),GRB.MINIMIZE)

modelf.optimize()
valor_optimo = modelf.objVal

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2050 rows, 100050 columns and 200050 nonzeros
Model fingerprint: 0xded8c7ec
Variable types: 0 continuous, 100050 integer (100050 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9951.0000000
Presolve time: 0.34s
Presolved: 2050 rows, 100050 columns, 200050 nonzeros
Variable types: 0 continuous, 100050 integer (100050 binary)

Root relaxation: objective 6.010960e+03, 5756 iterations, 0.36 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6010.96000    0   25 9951.00000 6010.96000  39.6%     -    1s
H    0     0                    6382.0000000 6010.96000  5.81%  

In [6]:
# Esta es la definición de variables y restricciones del problema completo original
xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")


modelf.setObjective((sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M)),GRB.MINIMIZE)

modelf.optimize()
valor_optimo = modelf.objVal

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2050 rows, 100050 columns and 200050 nonzeros
Model fingerprint: 0xded8c7ec
Variable types: 0 continuous, 100050 integer (100050 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9951.0000000
Presolve time: 0.39s
Presolved: 2050 rows, 100050 columns, 200050 nonzeros
Variable types: 0 continuous, 100050 integer (100050 binary)

Root relaxation: objective 6.010960e+03, 5756 iterations, 0.38 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6010.96000    0   25 9951.00000 6010.96000  39.6%     -    1s
H    0     0                    6382.0000000 6010.96000  5.81%  

In [16]:
# Aquí se declara el problema relajado lineal y se resuelve
modelr = modelf.relax()
modelr.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2050 rows, 100050 columns and 200050 nonzeros
Model fingerprint: 0xdc882382
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2050 rows and 100050 columns
Presolve time: 0.18s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0109600e+03   0.000000e+00   0.000000e+00      0s

Solved with dual simplex
Solved in 0 iterations and 0.23 seconds
Optimal objective  6.010960000e+03


In [17]:
# Se rescatan las variables duales del problema relajado lineal
duals = [c.Pi for c in modelr.getConstrs()]

In [18]:
#  Se definen las variables y restricciones  de la función dual
x = model.addVars(m,n,vtype=GRB.BINARY)
y = model.addVars(M,vtype=GRB.BINARY)

rest_rel = model.addConstrs(((sum(x[i,j] for j in N) <= n*y[i]) for i in M),name = "ASSIGN_REL")
model.update()

# se calculan las variables duales iniciales
u = [0.0]*n
for j in N:
    u[j] = duals[j]

u_cota = 10000 # herística Gurobi 

In [19]:
begin_time = datetime.datetime.now()


best = 0
best_cota = 0
# Este es el ciclo principal de la relajación lagrangeana
for k in range(1, NITER):
#   Se define la función objetivo de la función dual para esta iteración
    
    model.setObjective((sum(sum((c[i,j]+u[j])*x[i,j] for i in M) for j in N)+sum(f[i]*y[i] for i in M)),GRB.MINIMIZE)

    model.update()
    model.optimize()
    
#   Se calcula el valor de la función dual.

    dual_value = model.objVal - sum(u[j] for j in N)

    print( 'iteration', k, 'obj =', dual_value)
    
#   Se calcula el subgradiente a usar

    sgrad = [0.0]*n
    for j in N:
        sgrad[j] = sum(x[i,j].X for i in M) - 1
    
#   Acá se define el paso

#    s = 1.0/k
    rho = 1
    s = rho*abs(u_cota - dual_value)/(np.linalg.norm(sgrad)**2)

#   Se hace el avance en la dirección del subgradiente

    for j in N:
        u[j] = u[j] + s*(sgrad[j])
    
    
    x_valido = valid_solution()
    u_cota = upper_bound(x_valido)
    
    if dual_value > best: 
        best = dual_value
        best_cota = u_cota


print( '\n Objectivo modelo relajado =', modelr.objVal,'\n')
print( '\n Objectivo modelo completo =', valor_optimo,'\n')
print( '\n Valor dual alcanzado =', best,'\n')

print(f'Tiempo de ejecución: {datetime.datetime.now() - begin_time}')


iteration 1 obj = -6011.00850000016
iteration 2 obj = -63919.428903997206
iteration 3 obj = -15787.213350978003
iteration 4 obj = 930.9066559311959
iteration 5 obj = 5057.144211396406
iteration 6 obj = 5849.281822005943
iteration 7 obj = 5014.9400919206555
iteration 8 obj = 5802.384524491985
iteration 9 obj = 5680.933708095993
iteration 10 obj = 5729.071930204701
iteration 11 obj = 3417.1053547051724
iteration 12 obj = 5805.531139278859
iteration 13 obj = -3778.927598775109
iteration 14 obj = 5185.6749753452295
iteration 15 obj = 5810.946902283809
iteration 16 obj = -3471.168270756146
iteration 17 obj = 4440.537918378281
iteration 18 obj = 5811.547814663112
iteration 19 obj = 5783.376642129991
iteration 20 obj = 5775.591700312542
iteration 21 obj = 3802.457369172669
iteration 22 obj = 5799.0660850994
iteration 23 obj = -3913.482888688597
iteration 24 obj = 5552.900138967213
iteration 25 obj = 5824.377765482458
iteration 26 obj = -2695.4956152801587
iteration 27 obj = 5338.055775532881


In [21]:
print(f'Mejor cota superior: {best_cota}')

Mejor cota superior: 6347.0
